In [ ]:
!pip install pydub

In [ ]:
!pip install librosa

In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [ ]:
import sys 
import librosa
import numpy as np
import json
import pydub
from scipy.io import wavfile


In [ ]:
y,sr = librosa.load('test.wav',sr = 48000)
for i in range(5):
  y = np.concatenate((y,y),axis=0)
wavfile.write('test1.wav',sr,y[:sr*4698])

In [ ]:
fs = 16000
names = ['speaker1','speaker2','speaker3','speaker4','speaker5','speaker6','speaker7','speaker8','speaker9']

f = open('ResultAnalysis.json','r')
Jsondata = json.load(f)

for i in names:
    globals()[i] = np.empty([0, ])
dictionary = {}
for i in names:
  dictionary[i] = []


In [ ]:
dictionary['speaker1']

In [ ]:
cnt = 0
while cnt < len(Jsondata['speaker']):
  seg = Jsondata['speaker'][cnt]['Segment_type']
  conf = Jsondata['speaker'][cnt]['confidence_level']
  if(seg != 'music' and seg != 'silence' and conf == 'Confident'):
    dictionary[seg].append(cnt)
  cnt += 1

In [ ]:
data,sr = librosa.load('test1.wav',sr = 48000)


In [ ]:
for speaker in names:
  j = 0
  for i in range(len(dictionary[speaker])):
    cnt = dictionary[speaker][i]
    times = cnt - j
    if(times == 0):
      globals()[speaker] = np.concatenate((globals()[speaker],data[cnt*fs:(cnt+1)*fs]),axis=0)
      continue
    globals()[speaker] = np.concatenate((globals()[speaker], np.zeros((fs,times)).flatten()),axis=0)
    globals()[speaker] = np.concatenate((globals()[speaker],data[cnt*fs:(cnt+1)*fs]),axis=0)
    j = cnt+1
  if(j < 4698):
    times = 4698 - j
    globals()[speaker] = np.concatenate((globals()[speaker], np.zeros((fs,times)).flatten()),axis=0)
  break

In [ ]:
def write(f,sr,x,normalized=True):
  print('starting')
  channels = 2 if(x.ndim == 2 and x.shape[1] == 2) else 1
  if normalized:
    y = np.int16(x*2 **15)
  else:
    y = np.int16(x)
  song = pydub.AudioSegment(y.tobytes(),frame_rate = sr, sample_width = 2, channels = channels)
  song.export(f, format = "mp3", bitrate = "320k")
  print('end')


In [ ]:

from tqdm import tqdm
for i in tqdm(names):
  audioDATA = globals()[i]
  audioDATA = audioDATA.astype('float32')
  if max(audioDATA)==1:
    audioDATA = 32767*audioDATA
  else:
    audioDATA = 32768*audioDATA
  audioDATA = audioDATA.astype('int16')
  write('folder'  + '/' + i +'.mp3',fs,globals()[i])
  break

  0%|          | 0/9 [00:00<?, ?it/s]

starting
end


In [ ]:
string = ""
j=1
for i in range(len(dictionary['speaker1'])):
  if(j == dictionary['speaker1'][i]):
    continue
  string += 'volume=enable=\'between(t,'
  string += str(j) +','+ str(dictionary['speaker1'][i])
  string += ')\':volume=0, '
  j = dictionary['speaker1'][i]+1

string += 'volume=enable=\'between(t,'
string += str(j) +','+ str(4698)
string += ')\':volume=0, '
j = dictionary['speaker1'][i]+1

In [ ]:
string

"volume=enable='between(t,1,8)':volume=0, volume=enable='between(t,9,15)':volume=0, volume=enable='between(t,16,23)':volume=0, volume=enable='between(t,24,25)':volume=0, volume=enable='between(t,26,27)':volume=0, volume=enable='between(t,28,29)':volume=0, volume=enable='between(t,30,33)':volume=0, volume=enable='between(t,34,37)':volume=0, volume=enable='between(t,38,39)':volume=0, volume=enable='between(t,40,43)':volume=0, volume=enable='between(t,44,48)':volume=0, volume=enable='between(t,49,61)':volume=0, volume=enable='between(t,62,86)':volume=0, volume=enable='between(t,87,89)':volume=0, volume=enable='between(t,90,91)':volume=0, volume=enable='between(t,92,93)':volume=0, volume=enable='between(t,94,111)':volume=0, volume=enable='between(t,112,125)':volume=0, volume=enable='between(t,126,128)':volume=0, volume=enable='between(t,129,130)':volume=0, volume=enable='between(t,131,136)':volume=0, volume=enable='between(t,137,138)':volume=0, volume=enable='between(t,139,140)':volume=0, 

In [ ]:
# !ffmpeg -i speaker1.mp3 -af string mod_speaker1.mp3

import subprocess
string = ""
j=1
for i in range(len(dictionary['speaker1'])):
  if(j == dictionary['speaker1'][i]):
    continue
  string += 'volume=enable=\'between(t,'
  string += str(j) +','+ str(dictionary['speaker1'][i])
  string += ')\':volume=0, '
  j = dictionary['speaker1'][i]+1

string += 'volume=enable=\'between(t,'
string += str(j) +','+ str(4698)
string += ')\':volume=0, '
j = dictionary['speaker1'][i]+1

status,output = subprocess.getstatusoutput(" ffmpeg -i speaker1.mp3 -af " + string + " mod_speaker1.mp3")
print (output)
print(status)

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena